In [1]:
import numpy as np
from numpy.fft import fft
from scipy.stats import skew, kurtosis 
import pandas as pd
import glob
import matplotlib.pyplot as plt
import os
import struct
import glob
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, accuracy_score


import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import struct
from numpy.fft import fft
from scipy.stats import skew, kurtosis
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

# Check if CUDA is available and set the device to GPU if it is, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class ComplexNumbersDataset(Dataset):
    def __init__(self, filepath, sequence_length, max_samples=None, for_training=True, process_method='data1'):
        self.filepath = filepath
        self.sequence_length = sequence_length
        self.max_samples = max_samples
        self.for_training = for_training
        self.process_method = process_method
        self.samples = self.load_samples()
        self.samples_per_sequence = self.sequence_length
        #self.samples_per_sequence = self.sequence_length if for_training else 1

    def load_samples(self):
        samples = []
        with open(self.filepath, 'rb') as binary_file:
            while True:
                if self.max_samples and len(samples) >= self.max_samples:
                    break
                binary_data = binary_file.read(8)
                if not binary_data:
                    break
                decoded_data = struct.unpack('ff', binary_data)
                if decoded_data[0] == 0 and decoded_data[1] == 0:
                    continue
                samples.append(f"{decoded_data[0]}+{decoded_data[1]}j")
        return samples

    def process_data1(self, sequence_samples):
        #print('sequence_samples:', sequence_samples)
        #print('len(sequence_samples):', len(sequence_samples))
        real_parts = []
        imag_parts = []
        for sample in sequence_samples:
            # Remove potential unwanted characters (spaces, etc.)
            sample = sample.replace(" ", "").replace("+-", "-")
            try:
                c = complex(sample)
                real_parts.append(c.real)
                imag_parts.append(c.imag)
            except ValueError:
                print(f"Failed to convert: {sample}")  # This will show which string failed
                raise
        real_parts = np.array(real_parts)
        imag_parts = np.array(imag_parts)
        # Normalize
        real_parts = (real_parts - np.mean(real_parts)) / np.std(real_parts)
        imag_parts = (imag_parts - np.mean(imag_parts)) / np.std(imag_parts)

        # Combining real and imaginary parts
        X = np.stack((real_parts, imag_parts), axis=1)  # Shape: (sequence_length, 2)
        return torch.tensor(X, dtype=torch.float32)

    def process_data2(self, sequence_samples):
        samples_array = np.array([complex(sample) for sample in sequence_samples], dtype=np.complex64)
        samples_fft = fft(samples_array)
        real_parts = np.real(samples_fft)
        imag_parts = np.imag(samples_fft)
        
        # Normalization
        epsilon = 1e-10
        real_parts_normalized = (real_parts - np.mean(real_parts)) / (np.std(real_parts) + epsilon)
        imag_parts_normalized = (imag_parts - np.mean(imag_parts)) / (np.std(imag_parts) + epsilon)
        
        # Feature extraction
        features = np.column_stack((
            np.mean(real_parts_normalized), np.std(real_parts_normalized), skew(real_parts_normalized),
            kurtosis(real_parts_normalized), np.mean(imag_parts_normalized), np.std(imag_parts_normalized),
            skew(imag_parts_normalized), kurtosis(imag_parts_normalized)
        ))
        
        return torch.tensor(features, dtype=torch.float32).reshape(self.samples_per_sequence, -1)

    def __len__(self):
        return len(self.samples) // self.samples_per_sequence

    def __getitem__(self, idx):
        start_idx = idx * self.samples_per_sequence
        end_idx = start_idx + self.samples_per_sequence
        sequence_samples = self.samples[start_idx:end_idx]
        
        if self.process_method == 'data1':
            X = self.process_data1(sequence_samples)
        elif self.process_method == 'data2':
            X = self.process_data2(sequence_samples)
        else:
            raise ValueError("Invalid process method specified.")
        if self.for_training:
            return X, X
        else:
            return X
#------------------------------------------------------------------------------------------------------
class SelfAttentionLayer(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(SelfAttentionLayer, self).__init__()
        self.multihead_attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=num_heads)

    def forward(self, inputs):
        # Expecting inputs of shape (sequence_length, batch_size, embed_size)
        attn_output, _ = self.multihead_attention(inputs, inputs, inputs)
        # Output shape is the same as input shape (sequence_length, batch_size, embed_size)
        return attn_output

class LSTMAutoencoder(nn.Module):
    def __init__(self, sequence_length, input_dim, lstm_dim=50, num_heads=2):
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.lstm_dim = lstm_dim
        
        # Encoder
        self.encoder_lstm1 = nn.LSTM(input_dim, lstm_dim, batch_first=True)
        self.self_attention = SelfAttentionLayer(embed_size=lstm_dim, num_heads=num_heads)
        self.encoder_lstm2 = nn.LSTM(lstm_dim, lstm_dim, batch_first=True)
        
        # Decoder
        self.decoder_lstm1 = nn.LSTM(lstm_dim, lstm_dim, batch_first=True)
        self.decoder_lstm2 = nn.LSTM(lstm_dim, lstm_dim, batch_first=True)
        self.decoder_linear = nn.Linear(lstm_dim, input_dim)
        
    def forward(self, x):
        # Encoder
        x, (hidden_state, cell_state) = self.encoder_lstm1(x)
        # Permute x to match MultiheadAttention input shape expectations
        x = x.permute(1, 0, 2)  # Shape: (sequence_length, batch_size, embed_size)
        x = self.self_attention(x)
        # Permute x back to match LSTM input shape expectations
        x = x.permute(1, 0, 2)  # Shape: (batch_size, sequence_length, embed_size)
        x, (hidden_state, cell_state) = self.encoder_lstm2(x)
        
        # Decoder
        x, (hidden_state, cell_state) = self.decoder_lstm1(x)
        x, (hidden_state, cell_state) = self.decoder_lstm2(x)
        # Apply Linear layer to each time step
        x = self.decoder_linear(x)
        
        return x

# Example of initializing the model
sequence_length = 10

feature_dim = 2
lstm_dim = 50
num_heads = 2
epsilon_std = 0.1
model = LSTMAutoencoder(sequence_length=10, input_dim=feature_dim).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
loss_function = nn.MSELoss()


# Model Training
batch_size = 20
max_train_samples = 25000
train_steps = max_train_samples // (batch_size * sequence_length)
max_samples = 25000  # Maximum samples to read (or None to read all)
max_test_samples = 25000

# pure_file_pattern = '/home/mreza/5G accelerator/ID_MEC/data generator/pure_data/pure_iq_samples_*.csv'
# mixed_file_pattern = '/home/mreza/5G accelerator/ID_MEC/data generator/mixed_data/mixed_iq_samples_*.csv'
# pure_file_new = '/home/mreza/5G accelerator/ID_MEC/data generator/New Data-Collection/rx_IQ_pure'
# mixed_file_new = '/home/mreza/5G accelerator/ID_MEC/data generator/New Data-Collection/rx_IQ_MIX'
# pure_file_old = '/home/mreza/5G accelerator/IQ_samples/data collected/5G_DL_IQ_no_jamming_0924.dat'
# mixed_file_old = '/home/mreza/5G accelerator/IQ_samples/data collected/5G_DL_IQ_with_periodic_jamming_0928_02.dat'


pure_file_new = r"C:\Users\Mohammadreza\Desktop\My Class\Proj-DC\My Works\My Papers\intrusion\data generator\New Data-Collection\rx_IQ_pure"
mixed_file_new = r"C:\Users\Mohammadreza\Desktop\My Class\Proj-DC\My Works\My Papers\intrusion\data generator\New Data-Collection\rx_IQ_MIX"


# Creating dataset instances for training and validation/testing
train_gen_instance  = ComplexNumbersDataset(filepath=pure_file_new, 
                                            sequence_length=sequence_length, 
                                            max_samples=max_train_samples, for_training=True, 
                                            process_method='data1')

combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
                                              sequence_length=sequence_length, 
                                              max_samples=max_samples, for_training=False, 
                                              process_method='data1')

# Creating DataLoader instances for batching
train_loader = DataLoader(train_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)
combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


# Custom training loop in PyTorch
num_epochs = 2
for epoch in range(num_epochs):
    print(f"Using device: {device}")
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()  # Set the model to training mode
    
    total_loss = 0
    for batch_idx, (X_chunk, _) in enumerate(train_loader):  # Assuming autoencoder where input = target
        X_chunk = X_chunk.to(device)
        
        optimizer.zero_grad()
        reconstructed_x = model(X_chunk)
        #print('X_chunk.shape:', X_chunk.shape)
        #print('reconstructed_x.shape:', reconstructed_x.shape)
        loss = loss_function(reconstructed_x, X_chunk)  # Use X_chunk as both input and target
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}\n")






Using device: cpu
LSTMAutoencoder(
  (encoder_lstm1): LSTM(2, 50, batch_first=True)
  (self_attention): SelfAttentionLayer(
    (multihead_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (encoder_lstm2): LSTM(50, 50, batch_first=True)
  (decoder_lstm1): LSTM(50, 50, batch_first=True)
  (decoder_lstm2): LSTM(50, 50, batch_first=True)
  (decoder_linear): Linear(in_features=50, out_features=2, bias=True)
)
Using device: cpu
Epoch 1/2
Batch 0/125, Loss: 1.0080034732818604
Batch 100/125, Loss: 0.2216547578573227
Epoch 1, Average Loss: 0.5969463142156601

Using device: cpu
Epoch 2/2
Batch 0/125, Loss: 0.8442041277885437
Batch 100/125, Loss: 0.17711521685123444
Epoch 2, Average Loss: 0.3579025698304176



In [2]:

max_samples = 2500000  # Maximum samples to read (or None to read all)
max_test_samples = 2500000
combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
                                              sequence_length=sequence_length, 
                                              max_samples=max_samples, for_training=False, 
                                              process_method='data1')

# Creating DataLoader instances for batching
combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


#num_predictions = 100  # or any other large number
#print(f"Number of predictions to be performed: {num_predictions}")
reconstruction_errors = []
all_X_chunk_test = []
all_X_chunk_pred = []
all_intrusion_flags = []

accumulated_real_chunk_errors = []
accumulated_X_chunk_test = []

threshold = None
#---------------------------------------------------------------------------------------------------
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from itertools import islice

# Your existing code for dataset setup and model definition...

# Initialize the Jupyter Dash app
app = JupyterDash(__name__)

# Define the layout of the app
# app.layout = html.Div([
#     html.H1("Real-Time RNN Autoencoder Data and Prediction Visualization"),
#     dcc.Graph(id='live-data-feed'),
#     dcc.Graph(id='live-prediction'),
#     dcc.Graph(id='live-intrusion-detection'),
#     dcc.Interval(
#         id='interval-component',
#         interval=4*1000,  # Update every 2 seconds
#         n_intervals=0
#     )
# ])

app.layout = html.Div([
    html.H1("Real-Time RNN Autoencoder Data and Prediction Visualization"),
    dcc.Graph(id='live-data-feed'),        
    dcc.Graph(id='live-comparison'),
    dcc.Graph(id='accumulated-live-data-feed'),
    dcc.Graph(id='live-intrusion-detection'),
    dcc.Graph(id='live-prediction'),
    dcc.Interval(
        id='interval-component',
        interval=10,
        n_intervals=0
    )
])

current_prediction_count = 0
num_predictions = 500  # Set this to the desired number of predictions
data_iterator = iter(combined_loader)  # Create an iterator for the DataLoader

def get_new_data():
    global data_iterator, current_prediction_count, num_predictions, combined_loader, model, device
    model.eval()  # Ensure the model is in evaluation mode
    if current_prediction_count >= num_predictions:
        print("Reached the maximum number of predictions, stopping updates.")
        return None, None, None  # Will cause the callback to prevent an update
    
    try:
        # Fetch the next batch of data
        X_chunk_test = next(data_iterator).to(device)
        X_chunk_pred = model(X_chunk_test).detach().cpu()
        chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1).cpu().numpy()
        X_chunk_test = X_chunk_test.cpu().numpy()
        X_chunk_pred = X_chunk_pred.cpu().numpy()
        current_prediction_count += 1  # Increment the count after a successful prediction
    except StopIteration:
        # If we've reached the end of the dataset before reaching num_predictions, reset the iterator
        data_iterator = iter(combined_loader)  # Reset the iterator
        current_prediction_count = 0  # Reset the prediction count
        return get_new_data()  # Retry fetching data recursively

    return X_chunk_test, X_chunk_pred, chunk_errors

# Callback for the data feed graph to plot only the real part
@app.callback(Output('live-data-feed', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_data_feed(n):
    X_chunk_test, _, _ = get_new_data()
    print('X_chunk_test.shape:', X_chunk_test.shape)
    if X_chunk_test is None:
        raise dash.exceptions.PreventUpdate
    
    # Flatten the array and select only the real part (every second value starting from index 0)
    real_X_chunk_test = X_chunk_test[:,:,0].flatten()
    img_X_chunk_test = X_chunk_test[:,:,0].flatten()
    print('real_X_chunk_test.shape:', real_X_chunk_test.shape)
    print('img_X_chunk_test.shape:', img_X_chunk_test.shape)
    
    fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list(range(len(X_chunk_test.flatten()))), y=X_chunk_test.flatten(), mode='lines', name='Input Data'))

    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_test))),
        y=real_X_chunk_test,
        mode='lines',
        name='Input Data',
        line=dict(color='blue')
    ))

    fig.update_layout(
        title='Live Data Feed',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        )
    )
    return fig



# Callback for the prediction graph to plot only the real part
@app.callback(Output('live-prediction', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_prediction(n):
    _, X_chunk_pred, _ = get_new_data()
    if X_chunk_pred is None:
        raise dash.exceptions.PreventUpdate

    # Flatten the array and select only the real part (every second value starting from index 0)
    real_X_chunk_pred = X_chunk_pred[:,:,0].flatten()
    img_X_chunk_pred = X_chunk_pred[:,:,0].flatten()
    print('real_X_chunk_pred.shape:', real_X_chunk_pred.shape)
    print('img_X_chunk_pred.shape:', img_X_chunk_pred.shape)

    fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list(range(len(X_chunk_pred.flatten()))), y=X_chunk_pred.flatten(), mode='lines', name='Predicted Data'))

    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_pred))),
        y=real_X_chunk_pred,
        mode='lines',
        name='Predicted Data - Real Part',
        #line=dict(dash='dash', color='green')
        line=dict(color='green') 
    ))
    fig.update_layout(
        title='Live Prediction',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20), 
        )
    )
    return fig


# # Callback for the intrusion detection graph
# @app.callback(Output('live-intrusion-detection', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_intrusion_detection(n):
#     _, X_chunk_pred, chunk_errors = get_new_data()

#     if chunk_errors is None or not chunk_errors.any():
#         print("Chunk errors data is not valid, preventing update.")
#         raise dash.exceptions.PreventUpdate

#     max_chunk_errors = chunk_errors.max(axis=1)
#     real_chunk_errors = chunk_errors[:,0]

#     # Generate the x-axis labels to match the sequence indices
#     #sequence_indices = np.arange(max_chunk_errors.shape[0])
#     sequence_indices = np.arange(len(real_chunk_errors))
    
#     fig_intrusion_detection = go.Figure()
#     fig_intrusion_detection.add_trace(go.Scatter(
#         x=sequence_indices,
#         y=real_chunk_errors,
#         mode='lines+markers',
#         name='Real Reconstruction Error per Sequence',
#         line=dict(color='blue')
#     ))
#     fig_intrusion_detection.update_layout(
#         title='Live Intrusion Detection',
#         xaxis_title='Sequence Index',
#         yaxis_title='Real Reconstruction Error',
#         yaxis_type='linear'  # Using a linear scale of error magnitudes can use log
#     )
#     return fig_intrusion_detection




@app.callback(Output('live-intrusion-detection', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_intrusion_detection(n):
    global accumulated_real_chunk_errors, threshold
    _, _, chunk_errors = get_new_data()

    if chunk_errors is None or not chunk_errors.any():
        raise dash.exceptions.PreventUpdate
    
    real_chunk_errors = chunk_errors[:, 0]
    accumulated_real_chunk_errors.extend(real_chunk_errors)
    
    if threshold is None:
        # Assuming the first batch is representative for setting the threshold
        threshold = np.percentile(accumulated_real_chunk_errors, 98)
        print("Threshold set to:", threshold)

    sequence_indices = list(range(len(accumulated_real_chunk_errors)))
    
    fig_intrusion_detection = go.Figure()
    fig_intrusion_detection.add_trace(go.Scatter(
        x=sequence_indices,
        y=accumulated_real_chunk_errors,
        mode='lines+markers',
        name='Real Reconstruction Error per Sequence',
        line=dict(color='gray')
    ))

    # Add the constant threshold line
    fig_intrusion_detection.add_hline(y=threshold, line_dash="dot", line_color="red", line_width=3,
                                      #annotation_text=f"Threshold: {threshold:.2f}",
                                      annotation_text=f"Threshold",
                                      annotation_position="top right",
                                      annotation_font=dict(size=14))

    fig_intrusion_detection.update_layout(
        title='Live Intrusion Detection',
        title_font=dict(size=28),
        xaxis_title='Sequence Index',
        xaxis=dict(
            title='Sequence Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Reconstruction Error',
        yaxis_type='linear',  # or 'log'
        yaxis=dict(
            title='Real Reconstruction Error',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        )
    )
    return fig_intrusion_detection

# Callback for the per-batch comparison graph
@app.callback(Output('live-comparison', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_comparison_graph(n):
    X_chunk_test, X_chunk_pred, _ = get_new_data()
    if X_chunk_test is None or X_chunk_pred is None:
        raise dash.exceptions.PreventUpdate
    
    # Extract both real parts for comparison
    real_X_chunk_test = X_chunk_test[:, :, 0].flatten()
    real_X_chunk_pred = X_chunk_pred[:, :, 0].flatten()

    fig = go.Figure()

    # Plot the real part of the test data
    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_test))),
        y=real_X_chunk_test,
        mode='lines',
        name='Test Data - Real Part',
        line=dict(color='blue')
    ))

    # Plot the real part of the prediction data
    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_pred))),
        y=real_X_chunk_pred,
        mode='lines',
        name='Prediction Data - Real Part',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title='Data Prediction Per Batch',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        legend=dict(
            font=dict(size=20)
        )
    )

    return fig
    
# Callback for the accumulated live data feed graph
@app.callback(Output('accumulated-live-data-feed', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_accumulated_data_feed(n):
    global accumulated_X_chunk_test
    X_chunk_test, _, _ = get_new_data()
    if X_chunk_test is None:
        raise dash.exceptions.PreventUpdate

    # Flatten the test data and accumulate it
    real_X_chunk_test = X_chunk_test[:,:,0].flatten()  # Assuming you only want the real part
    accumulated_X_chunk_test.extend(real_X_chunk_test)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(len(accumulated_X_chunk_test))),
        y=accumulated_X_chunk_test,
        mode='lines',
        name='Accumulated Input Data',
        line=dict(color='purple')
    ))
    fig.update_layout(
        title='Accumulated Live Data Feed',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        )
    )
    return fig


# Run the app
#app.run_server(mode='external')
app.run_server(mode='external', debug=False)

#-----------------------------------------------------------------------------------------------------------

C:\Users\Mohammadreza\AppData\Local\Programs\Python\Python310\lib\site-packages\dash\dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
Threshold set to: 1.6949256229400635
Dash app running on http://127.0.0.1:8050/
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img

real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.sh

X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
Reached the maximum number of predictions, stopping updates.
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
Reached the maximum number of predi

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

...

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.Reached the maximum number of predictions, stopping updates.

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

In [ ]:

# max_samples = 1000000  # Maximum samples to read (or None to read all)
# max_test_samples = 1000000
# combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
#                                               sequence_length=sequence_length, 
#                                               max_samples=max_samples, for_training=False, 
#                                               process_method='data1')

# # Creating DataLoader instances for batching
# combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


# #num_predictions = 100  # or any other large number
# #print(f"Number of predictions to be performed: {num_predictions}")
# reconstruction_errors = []
# all_X_chunk_test = []
# all_X_chunk_pred = []
# all_intrusion_flags = []

# accumulated_real_X_chunk_test = []
# accumulated_real_X_chunk_pred = []
# accumulated_real_chunk_errors = []
# # Initialize the threshold variable
# threshold = None

# #---------------------------------------------------------------------------------------------------
# from jupyter_dash import JupyterDash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import plotly.graph_objs as go
# import torch
# from torch.utils.data import DataLoader
# from itertools import islice

# # Your existing code for dataset setup and model definition...

# # Initialize the Jupyter Dash app
# app = JupyterDash(__name__)

# # Define the layout of the app
# app.layout = html.Div([
#     html.H1("Real-Time Enhanced Self-Attention RNN Autoencoder Visualization"),
#     dcc.Graph(id='live-data-feed'),
#     dcc.Graph(id='live-prediction'),    
#     dcc.Graph(id='live-intrusion-detection'),
#     dcc.Graph(id='live-prediction-per-batch'),
#     dcc.Interval(
#         id='interval-component',
#         interval=4*1000,  # Update every 2 seconds
#         n_intervals=0
#     )
# ])


# current_prediction_count = 0
# num_predictions = 200  # Set this to the desired number of predictions
# data_iterator = iter(combined_loader)  # Create an iterator for the DataLoader

# def get_new_data():
#     global data_iterator, current_prediction_count, num_predictions, combined_loader, model, device
#     model.eval()  # Ensure the model is in evaluation mode
#     if current_prediction_count >= num_predictions:
#         print("Reached the maximum number of predictions, stopping updates.")
#         return None, None, None  # Will cause the callback to prevent an update
    
#     try:
#         # Fetch the next batch of data
#         X_chunk_test = next(data_iterator).to(device)
#         X_chunk_pred = model(X_chunk_test).detach().cpu()
#         chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1).cpu().numpy()
#         X_chunk_test = X_chunk_test.cpu().numpy()
#         X_chunk_pred = X_chunk_pred.cpu().numpy()
#         current_prediction_count += 1  # Increment the count after a successful prediction
#     except StopIteration:
#         # If we've reached the end of the dataset before reaching num_predictions, reset the iterator
#         data_iterator = iter(combined_loader)  # Reset the iterator
#         current_prediction_count = 0  # Reset the prediction count
#         return get_new_data()  # Retry fetching data recursively

#     return X_chunk_test, X_chunk_pred, chunk_errors

# # Callback for the data feed graph to plot only the real part
# @app.callback(Output('live-data-feed', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_data_feed(n):
#     global accumulated_real_X_chunk_test
#     X_chunk_test, _, _ = get_new_data()
#     if X_chunk_test is None:
#         raise dash.exceptions.PreventUpdate

#     # Extract the real part and flatten it
#     real_X_chunk_test = X_chunk_test[:,:,0].flatten()
#     accumulated_real_X_chunk_test.extend(real_X_chunk_test)  # Accumulate the data

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(
#         x=list(range(len(accumulated_real_X_chunk_test))),
#         y=accumulated_real_X_chunk_test,
#         mode='lines',
#         name='Input Data',
#         line=dict(color='blue')
#     ))
#     fig.update_layout(title='Live Data Feed', xaxis_title='Sample Index', yaxis_title='Value')
#     return fig


# # Callback for the prediction graph to plot only the real part
# @app.callback(Output('live-prediction', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_prediction(n):
#     global accumulated_real_X_chunk_pred
#     _, X_chunk_pred, _ = get_new_data()
#     if X_chunk_pred is None:
#         raise dash.exceptions.PreventUpdate
    
#     # Extract the real part and flatten it
#     real_X_chunk_pred = X_chunk_pred[:, :, 0].flatten()
#     accumulated_real_X_chunk_pred.extend(real_X_chunk_pred)  # Accumulate the data
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(
#         x=list(range(len(accumulated_real_X_chunk_pred))),
#         y=accumulated_real_X_chunk_pred,
#         mode='lines',
#         name='Predicted Data - Real Part',
#         #line=dict(color='green', dash='dash')
#         line=dict(color='green')
#     ))
#     fig.update_layout(title='Live Prediction', xaxis_title='Sample Index', yaxis_title='Value')
#     return fig

# # Callback for the per-batch real part graph
# @app.callback(Output('live-prediction-per-batch', 'figure'),
#               [Input('interval-component', 'n_intervals')])
# def update_per_batch_graph(n):
#     X_chunk_test, X_chunk_pred, _ = get_new_data()
#     if X_chunk_test is None or X_chunk_pred is None:
#         raise dash.exceptions.PreventUpdate

#     # Extract the real part of the test and prediction data
#     real_X_chunk_test = X_chunk_test[:, :, 0].flatten()
#     real_X_chunk_pred = X_chunk_pred[:, :, 0].flatten()
#     img_X_chunk_test = X_chunk_test[:, :, 1].flatten()
#     img_X_chunk_pred = X_chunk_pred[:, :, 1].flatten()
    
#     fig = go.Figure()

#     # Plot the real part of the test data
#     fig.add_trace(go.Scatter(
#         x=list(range(len(img_X_chunk_test))),
#         y=img_X_chunk_test,
#         mode='lines',
#         name='Input Data',
#         line=dict(color='red')
#     ))

#     # Plot the real part of the prediction data
#     fig.add_trace(go.Scatter(
#         x=list(range(len(img_X_chunk_pred))),
#         y=img_X_chunk_pred,
#         mode='lines',
#         name='Predicted Data',
#         line=dict(color='green', dash='dash')
#     ))

#     fig.update_layout(title='Prediction Per Batch', xaxis_title='Sample Index', yaxis_title='Value')

#     return fig


# @app.callback(Output('live-intrusion-detection', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_intrusion_detection(n):
#     global accumulated_real_chunk_errors, threshold
#     _, _, chunk_errors = get_new_data()

#     if chunk_errors is None or not chunk_errors.any():
#         raise dash.exceptions.PreventUpdate
    
#     real_chunk_errors = chunk_errors[:, 0]
#     accumulated_real_chunk_errors.extend(real_chunk_errors)
    
#     if threshold is None:
#         # Assuming the first batch is representative for setting the threshold
#         threshold = np.percentile(accumulated_real_chunk_errors, 98)
#         print("Threshold set to:", threshold)

#     sequence_indices = list(range(len(accumulated_real_chunk_errors)))
    
#     fig_intrusion_detection = go.Figure()
#     fig_intrusion_detection.add_trace(go.Scatter(
#         x=sequence_indices,
#         y=accumulated_real_chunk_errors,
#         mode='lines+markers',
#         name='Real Reconstruction Error per Sequence',
#         line=dict(color='gray')
#     ))

#     # Add the constant threshold line
#     fig_intrusion_detection.add_hline(y=threshold, line_dash="dot", line_color="red", line_width=3,
#                                       #annotation_text=f"Threshold: {threshold:.2f}",
#                                       annotation_text=f"Threshold",
#                                       annotation_position="top right",
#                                       annotation_font=dict(size=14))

#     fig_intrusion_detection.update_layout(
#         title='Live Intrusion Detection',
#         xaxis_title='Sequence Index',
#         yaxis_title='Real Reconstruction Error',
#         yaxis_type='linear'  # or 'log'
#     )
#     return fig_intrusion_detection


# # Run the app
# #app.run_server(mode='external')
# app.run_server(mode='external', debug=False)

# #-----------------------------------------------------------------------------------------------------------

In [ ]:

# max_samples = 1000000  # Maximum samples to read (or None to read all)
# max_test_samples = 1000000
# combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
#                                               sequence_length=sequence_length, 
#                                               max_samples=max_samples, for_training=False, 
#                                               process_method='data1')

# # Creating DataLoader instances for batching
# combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


# #num_predictions = 100  # or any other large number
# #print(f"Number of predictions to be performed: {num_predictions}")
# reconstruction_errors = []
# all_X_chunk_test = []
# all_X_chunk_pred = []
# all_intrusion_flags = []

# #---------------------------------------------------------------------------------------------------
# from jupyter_dash import JupyterDash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import plotly.graph_objs as go
# import torch
# from torch.utils.data import DataLoader
# from itertools import islice

# # Your existing code for dataset setup and model definition...

# # Initialize the Jupyter Dash app
# app = JupyterDash(__name__)

# # Define the layout of the app
# app.layout = html.Div([
#     html.H1("Real-Time RNN Autoencoder Data and Prediction Visualization"),
#     dcc.Graph(id='live-data-feed'),
#     dcc.Graph(id='live-prediction'),
#     dcc.Graph(id='live-intrusion-detection'),
#     dcc.Interval(
#         id='interval-component',
#         interval=4*1000,  # Update every 2 seconds
#         n_intervals=0
#     )
# ])

# # Helper function to get new data
# # def get_new_data():
# #     global combined_loader, model, device
# #     model.eval()  # Ensure the model is in evaluation mode
# #     try:
# #         # Fetch the next batch of data
# #         X_chunk_test = next(iter(combined_loader)).to(device)
# #         print('X_chunk_test.shape:', X_chunk_test.shape)
# #         X_chunk_pred = model(X_chunk_test).detach().cpu()
# #         print('X_chunk_pred.shape:', X_chunk_pred.shape)
# #         chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1).cpu().numpy()
# #         print('chunk_errors.shape:', chunk_errors.shape)
# #         X_chunk_test = X_chunk_test.cpu().numpy()
# #         X_chunk_pred = X_chunk_pred.cpu().numpy()
# #     except StopIteration:
# #         # Reset the DataLoader if the end is reached
# #         combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)
# #         return None, None, None
# #     return X_chunk_test, X_chunk_pred, chunk_errors

# current_prediction_count = 0
# num_predictions = 100  # Set this to the desired number of predictions
# data_iterator = iter(combined_loader)  # Create an iterator for the DataLoader

# def get_new_data():
#     global data_iterator, current_prediction_count, num_predictions, combined_loader, model, device
#     model.eval()  # Ensure the model is in evaluation mode
#     if current_prediction_count >= num_predictions:
#         print("Reached the maximum number of predictions, stopping updates.")
#         return None, None, None  # Will cause the callback to prevent an update
    
#     try:
#         # Fetch the next batch of data
#         X_chunk_test = next(data_iterator).to(device)
#         X_chunk_pred = model(X_chunk_test).detach().cpu()
#         chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1).cpu().numpy()
#         X_chunk_test = X_chunk_test.cpu().numpy()
#         X_chunk_pred = X_chunk_pred.cpu().numpy()
#         current_prediction_count += 1  # Increment the count after a successful prediction
#     except StopIteration:
#         # If we've reached the end of the dataset before reaching num_predictions, reset the iterator
#         data_iterator = iter(combined_loader)  # Reset the iterator
#         current_prediction_count = 0  # Reset the prediction count
#         return get_new_data()  # Retry fetching data recursively

#     return X_chunk_test, X_chunk_pred, chunk_errors

# # Callback for the data feed graph
# @app.callback(Output('live-data-feed', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_data_feed(n):
#     X_chunk_test, _, _ = get_new_data()
#     print('X_chunk_test:', X_chunk_test)
#     if X_chunk_test is None:
#         raise dash.exceptions.PreventUpdate

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list(range(len(X_chunk_test.flatten()))), y=X_chunk_test.flatten(), mode='lines', name='Input Data'))
#     fig.update_layout(title='Live Data Feed', xaxis_title='Sample Index', yaxis_title='Value')
#     return fig

# # Callback for the prediction graph
# @app.callback(Output('live-prediction', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_prediction(n):
#     _, X_chunk_pred, _ = get_new_data()
#     print('X_chunk_pred:', X_chunk_pred)
#     if X_chunk_pred is None:
#         raise dash.exceptions.PreventUpdate

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list(range(len(X_chunk_pred.flatten()))), y=X_chunk_pred.flatten(), mode='lines', name='Predicted Data'))
#     fig.update_layout(title='Live Prediction', xaxis_title='Sample Index', yaxis_title='Value')
#     return fig

# # Callback for the intrusion detection graph
# # @app.callback(Output('live-intrusion-detection', 'figure'),
# #               Input('interval-component', 'n_intervals'))
# # def update_intrusion_detection(n):
# #     _, _, chunk_errors = get_new_data()
# #     print('chunk_errors.shape:', chunk_errors.shape)
# #     if chunk_errors is None:
# #         raise dash.exceptions.PreventUpdate

# #     fig = go.Figure()
# #     fig.add_trace(go.Scatter(x=list(range(len(chunk_errors))), y=chunk_errors, mode='lines', name='Reconstruction Error'))
# #     fig.update_layout(title='Live Intrusion Detection', xaxis_title='Sample Index', yaxis_title='Error Value')
# #     return fig

# # Callback for the intrusion detection graph
# @app.callback(Output('live-intrusion-detection', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_intrusion_detection(n):
#     _, _, chunk_errors = get_new_data()

#     if chunk_errors is None or not chunk_errors.any():
#         print("Chunk errors data is not valid, preventing update.")
#         raise dash.exceptions.PreventUpdate

#     # Compute the maximum error per sequence by taking the max across real and imaginary errors
#     max_chunk_errors = chunk_errors.max(axis=1)
#     print("Max Chunk errors received:", max_chunk_errors)  # Debug print

#     # Generate the x-axis labels to match the sequence indices
#     sequence_indices = np.arange(max_chunk_errors.shape[0])
    
#     fig_intrusion_detection = go.Figure()
#     fig_intrusion_detection.add_trace(go.Scatter(
#         x=sequence_indices,
#         y=max_chunk_errors,
#         mode='lines+markers',
#         name='Maximum Reconstruction Error per Sequence'
#     ))
#     fig_intrusion_detection.update_layout(
#         title='Live Intrusion Detection',
#         xaxis_title='Sequence Index',
#         yaxis_title='Maximum Reconstruction Error',
#         yaxis_type='linear'  # Using a linear scale to ensure visibility of error magnitudes
#     )
#     return fig_intrusion_detection




# # Run the app
# #app.run_server(mode='external')
# app.run_server(mode='external', debug=False)

# #-----------------------------------------------------------------------------------------------------------

In [ ]:

# combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
#                                               sequence_length=sequence_length, 
#                                               max_samples=max_samples, for_training=False, 
#                                               process_method='data1')

# # Creating DataLoader instances for batching
# combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


# num_predictions = 10  # or any other large number
# print(f"Number of predictions to be performed: {num_predictions}")
# reconstruction_errors = []
# all_X_chunk_test = []
# all_X_chunk_pred = []
# all_intrusion_flags = []

# # #-----------------------------------------------------------------------------------------

# from jupyter_dash import JupyterDash
# from dash import dcc, html
# from dash.dependencies import Input, Output, State
# import plotly.graph_objs as go
# import torch
# from torch.utils.data import DataLoader

# # existing code for dataset setup and model definition...

# # Initialize the Jupyter Dash app
# app = JupyterDash(__name__)

# # Define the layout of the app
# app.layout = html.Div([
#     html.H1("Real-Time RNN Autoencoder Data and Prediction Visualization"),
#     dcc.Graph(id='live-data-feed'),
#     dcc.Graph(id='live-prediction'),
#     dcc.Graph(id='live-intrusion-detection'),
#     dcc.Interval(
#         id='interval-component',
#         interval=2*1000,  # Update every 2 seconds
#         n_intervals=0
#     )
# ])

# # Helper function to get new data
# def get_new_data():
#     global combined_loader, model, device
#     model.eval()  # Ensure the model is in evaluation mode
#     try:
#         # Fetch the next batch of data
#         X_chunk_test = next(iter(combined_loader)).to(device)
#         X_chunk_pred = model(X_chunk_test).detach().cpu()
#         chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1).cpu().numpy()
#         X_chunk_test = X_chunk_test.cpu().numpy()
#         X_chunk_pred = X_chunk_pred.cpu().numpy()
#     except StopIteration:
#         # Reset the DataLoader if the end is reached
#         combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)
#         return None, None, None
#     return X_chunk_test, X_chunk_pred, chunk_errors

# # Callback for updating all three graphs simultaneously
# @app.callback(
#     [Output('live-data-feed', 'figure'),
#      Output('live-prediction', 'figure'),
#      Output('live-intrusion-detection', 'figure')],
#     [Input('interval-component', 'n_intervals')]
# )
# def update_graphs(n):
#     X_chunk_test, X_chunk_pred, chunk_errors = get_new_data()
#     print('X_chunk_test.shape:', X_chunk_test.shape)
#     print('X_chunk_pred.shape:', X_chunk_pred.shape)
#     print('chunk_errors.shape:', chunk_errors.shape)
#     if X_chunk_test is None or X_chunk_pred is None or chunk_errors is None:
#         raise dash.exceptions.PreventUpdate

#     # Figure for the data feed
#     fig_data_feed = go.Figure()
#     fig_data_feed.add_trace(go.Scatter(
#         x=list(range(len(X_chunk_test.flatten()))), 
#         y=X_chunk_test.flatten(), 
#         mode='lines', 
#         name='Input Data'))
#     fig_data_feed.update_layout(title='Live Data Feed', xaxis_title='Sample Index', yaxis_title='Value')

#     # Figure for the predictions
#     fig_prediction = go.Figure()
#     fig_prediction.add_trace(go.Scatter(
#         x=list(range(len(X_chunk_pred.flatten()))), 
#         y=X_chunk_pred.flatten(), 
#         mode='lines', 
#         name='Predicted Data'))
#     fig_prediction.update_layout(title='Live Prediction', xaxis_title='Sample Index', yaxis_title='Value')

#     # Figure for the intrusion detection
#     fig_intrusion_detection = go.Figure()
#     fig_intrusion_detection.add_trace(go.Scatter(
#         x=list(range(len(chunk_errors))), 
#         y=chunk_errors, 
#         mode='lines', 
#         name='Reconstruction Error'))
#     fig_intrusion_detection.update_layout(title='Live Intrusion Detection', xaxis_title='Sample Index', yaxis_title='Error Value')

#     return [fig_data_feed, fig_prediction, fig_intrusion_detection]

# # Run the app
# app.run_server(mode='external')
# # #------------------------------------------------------------------------------------------------------

In [ ]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for batch_idx, X_chunk_test in enumerate(combined_loader):
        X_chunk_test = X_chunk_test.to(device)
        X_chunk_pred = model(X_chunk_test)
        
        chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1)
        reconstruction_errors.extend(chunk_errors.cpu().numpy())
        all_X_chunk_test.append(X_chunk_test)
        all_X_chunk_pred.append(X_chunk_pred)


        if batch_idx % 100 == 0:
            print(f"Evaluating Batch {batch_idx}/{len(combined_loader)}")

reconstruction_error = np.array(reconstruction_errors)
print(f"Completed predictions. Total number of batches: {len(combined_loader)}")

#reconstruction_error = np.array(reconstruction_errors)
print('reconstruction_error.shape:', reconstruction_error.shape)
print('Number of NaNs in reconstruction_error:', np.isnan(reconstruction_error).sum())
max_error_per_sequence = reconstruction_error.max(axis=1) # Max error for each sequence
#print('max_error_per_sequence:', max_error_per_sequence)

print('max_error_per_sequence.shape:', max_error_per_sequence.shape)

threshold1 = np.percentile(max_error_per_sequence, 98)
print('threshold1:', threshold1)
threshold2 = np.percentile(reconstruction_error, 95)
print('threshold percentile:', threshold2)

is_intrusion_detected = max_error_per_sequence > threshold1  # Boolean array for sequences
print('len(is_intrusion_detected):', len(is_intrusion_detected))
print('is_intrusion_detected.shape:', is_intrusion_detected.shape)

#is_intrusion_detected2 = error_per_sequence > threshold1

num_total_sequences = len(max_error_per_sequence)
num_total_sequences2 = num_predictions * batch_size - num_predictions
print('num_total_sequences:', num_total_sequences)
print('num_total_sequences2:', num_total_sequences2)

#---------------------------------------finish 111-----------------------------------
flat_error_per_sequence = max_error_per_sequence.flatten()
#flat_error_per_sequence2 = error_per_sequence.flatten()
# Determine if intrusion detected for each sequence
for error in flat_error_per_sequence:
    all_intrusion_flags.append(error > threshold1)
all_X_chunk_test = [x.cpu() for x in all_X_chunk_test]
all_X_chunk_pred = [x.cpu() for x in all_X_chunk_pred]

all_X_chunk_test = np.concatenate(all_X_chunk_test, axis=0)
all_X_chunk_pred = np.concatenate(all_X_chunk_pred, axis=0)

#save_path = 'C:\\Users\\Mohammadreza\\Desktop\\My Class\\Proj-DC\\My Works\\My Papers\\intrusion\\data generator\\intrusion_detected'
#plot_with_intrusions8(all_X_chunk_test, all_X_chunk_pred, all_intrusion_flags, sequence_length, save_path)

jamming_detected = reconstruction_error > threshold1
#train_gen_instance.close()
#combined_gen_instance.close()
#Table
flattened_jamming_detected = jamming_detected.flatten()
real_part_detected = jamming_detected[:, 0]
imag_part_detected = jamming_detected[:, 1]

real_true_count = np.sum(real_part_detected)
real_false_count = len(real_part_detected) - real_true_count

imag_true_count = np.sum(imag_part_detected)
imag_false_count = len(imag_part_detected) - imag_true_count
# Overall
overall_true_count = np.sum(flattened_jamming_detected)
overall_false_count = len(flattened_jamming_detected) - overall_true_count
# Table-DataFrame
df = pd.DataFrame({
    'Part': ['Real', 'Imaginary', 'Overall'],
    'True Count': [real_true_count, imag_true_count, overall_true_count],
    'False Count': [real_false_count, imag_false_count, overall_false_count]
})
print(df)
num_jamming_detected = np.sum(jamming_detected)
print(f"Number of jamming sequences detected: {num_jamming_detected} out of {len(flattened_jamming_detected)} sequences")



In [ ]:
n = 4
sequence_index = np.random.choice(len(X_chunk_test) - n + 1)
# Ensure n sequences are available
n = min(n, len(X_chunk_test) - sequence_index)

# Convert PyTorch tensors to numpy arrays
original_sample_np = X_chunk_test[sequence_index:sequence_index + n].detach().cpu().numpy()
reconstructed_sample_np = X_chunk_pred[sequence_index:sequence_index + n].detach().cpu().numpy()

# Since your data is 3-dimensional (batch, sequence, features), you'll want to concatenate along the sequence axis (axis=1)
original_sample_concat = np.concatenate(original_sample_np, axis=0)
reconstructed_sample_concat = np.concatenate(reconstructed_sample_np, axis=0)

# Plot concatenated sequences
plt.figure(figsize=(14, 6))
plt.plot(original_sample_concat[:, 0], 'b-', label='Original Real Part')  # Assuming first feature is the real part
plt.plot(reconstructed_sample_concat[:, 0], 'r--', label='Reconstructed Real Part')  # Assuming first feature is the real part
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the number of sequences to plot together
n = 4  # Change this to desired number of sequences

# Ensure that we have enough samples for the desired number of sequences
sequence_index = np.random.choice(len(X_chunk_test) - n + 1)

# Convert PyTorch tensors to numpy arrays and concatenate selected sequences
original_sample_concat = np.concatenate(X_chunk_test[sequence_index:sequence_index + n].cpu().numpy(), axis=0)
reconstructed_sample_concat = np.concatenate(X_chunk_pred[sequence_index:sequence_index + n].cpu().numpy(), axis=0)

# Plot concatenated sequences for n = 4
plt.figure(figsize=(14, 6))
plt.plot(original_sample_concat[:, 0], 'b-', label='Original Real Part')
plt.plot(reconstructed_sample_concat[:, 0], 'r--', label='Reconstructed Real Part')
plt.plot(original_sample_concat[:, 1], 'y-', label='Original Imaginary Part')
plt.plot(reconstructed_sample_concat[:, 1], 'g--', label='Reconstructed Imaginary Part')
plt.title(f'Original vs Reconstructed IQ Data for {n} Sequences of Length {sequence_length}')
plt.xlabel('Time Steps')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

# Repeat for n = 2
n = 2  # Change this to a different desired number of sequences
sequence_index = np.random.choice(len(X_chunk_test) - n + 1)
original_sample_concat = np.concatenate(X_chunk_test[sequence_index:sequence_index + n].cpu().numpy(), axis=0)
reconstructed_sample_concat = np.concatenate(X_chunk_pred[sequence_index:sequence_index + n].cpu().numpy(), axis=0)

# Plot concatenated sequences for n = 2
plt.figure(figsize=(14, 6))
plt.plot(original_sample_concat[:, 0], 'b-', label='Original Real Part')
plt.plot(reconstructed_sample_concat[:, 0], 'r--', label='Reconstructed Real Part')
plt.plot(original_sample_concat[:, 1], 'y-', label='Original Imaginary Part')
plt.plot(reconstructed_sample_concat[:, 1], 'g--', label='Reconstructed Imaginary Part')
plt.title(f'Original vs Reconstructed IQ Data for {n} Sequences of Length {sequence_length}')
plt.xlabel('Time Steps')
plt.ylabel('Amplitude')
plt.legend()
plt.show()


In [ ]:

# reconstruction error
plt.figure(figsize=(14, 6))
plt.plot(reconstruction_error, label='Reconstruction Error')
plt.axhline(y=threshold2, color='r', linestyle='--', label='Threshold')
plt.title('Reconstruction Error with Threshold')
plt.xlabel('Sequence Number')
plt.ylabel('Reconstruction Error')
plt.legend()
# plt.savefig('1-Reconstruction Error with Threshold.png')
# plt.close()
plt.show()

In [ ]:
# Flatten the reconstruction_error to 1D
reconstruction_error_flat = reconstruction_error.flatten()
# reconstruction error
plt.figure(figsize=(14, 6))
plt.plot(reconstruction_error_flat, label='Reconstruction Error')
plt.axhline(y=threshold2, color='r', linestyle='--', label='Threshold')
plt.title('Reconstruction Error with Threshold')
plt.xlabel('Sequence Number')
plt.ylabel('Reconstruction Error')
plt.legend()
# plt.savefig('1-Reconstruction Error with Threshold.png')
# plt.close()
plt.show()


In [ ]:
#Histogram of Reconstruction Errors:
plt.figure(figsize=(14, 6))
plt.hist(reconstruction_error, bins=50, alpha=0.75)
plt.axvline(x=threshold2, color='r', linestyle='--', label='Threshold')
plt.title('Histogram of Reconstruction Errors')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.legend()
# plt.savefig('4-Histogram of Reconstruction Errors.png')
# plt.close()
plt.show()


In [ ]:
# Flatten the reconstruction_error to 1D
reconstruction_error_flat = reconstruction_error.flatten()
#Histogram of Reconstruction Errors:
plt.figure(figsize=(14, 6))
plt.hist(reconstruction_error_flat, bins=50, alpha=0.75)
plt.axvline(x=threshold2, color='r', linestyle='--', label='Threshold')
plt.title('Histogram of Reconstruction Errors')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.legend()
# plt.savefig('4-Histogram of Reconstruction Errors.png')
# plt.close()
plt.show()

In [ ]:
# Assuming X_chunk_test and X_chunk_pred are tensors on the GPU
# First, move the selected samples to CPU and convert to NumPy for plotting
sample_index = np.random.choice(len(X_chunk_test))
original_sample = X_chunk_test[sample_index].cpu().numpy()
reconstructed_sample = X_chunk_pred[sample_index].cpu().numpy()

plt.figure(figsize=(14, 6))
plt.plot(original_sample[:, 0], 'b-', label='Original Real Part')
plt.plot(reconstructed_sample[:, 0], 'r--', label='Reconstructed Real Part')
plt.plot(original_sample[:, 1], 'g-', label='Original Imaginary Part')
plt.plot(reconstructed_sample[:, 1], 'y--', label='Reconstructed Imaginary Part')
plt.title('Original vs Reconstructed IQ Data')
plt.xlabel('Time Steps')
plt.ylabel('Amplitude')
plt.legend()
plt.show()


In [ ]:
# Convert PyTorch tensors to NumPy arrays before calculating means
avg_real = np.mean(X_chunk_test.cpu().numpy(), axis=1)[:, 0]
avg_imag = np.mean(X_chunk_test.cpu().numpy(), axis=1)[:, 1]

# Assuming reconstruction_errors is a NumPy array or a list of errors
# If reconstruction_errors is a tensor, ensure to convert it with .cpu().numpy() as well
last_errors = np.mean(reconstruction_errors[-len(X_chunk_test):], axis=1)

print("Shape of avg_real:", avg_real.shape)
print("Shape of avg_imag:", avg_imag.shape)
print("Shape of last_errors:", last_errors.shape)

plt.figure(figsize=(14, 6))
plt.scatter(avg_real, last_errors, label='Real Part', alpha=0.5)
plt.axhline(y=threshold2, color='r', linestyle='--', label='Threshold')
plt.title('Reconstruction Error vs. Average Real Part')
plt.xlabel('Average Amplitude')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.show()

plt.figure(figsize=(14, 6))
plt.scatter(avg_imag, last_errors, label='Imaginary Part', alpha=0.5)
plt.axhline(y=threshold2, color='r', linestyle='--', label='Threshold')
plt.title('Reconstruction Error vs. Average Imaginary Part')
plt.xlabel('Average Amplitude')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.show()
